In [ ]:
#!pip install yfinance

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import requests
import re
from io import StringIO

import yfinance as yf
import pandas_datareader as pdr

import time
from datetime import date

## Question 1: [IPO] Withdrawn IPOs by Company Type

### What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

From the withdrawn IPO list (stockanalysis.com/ipos/withdrawn), collect and process the data to find out which company type saw the most withdrawn IPO value.

**Steps:**
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame. It is a similar process to Code Snippet 1 discussed at the livestream. You should get 99 entries.
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
  - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
  - “Inc” or “Incorporated” → `Inc`
  - “Group” → `Group`
  - “Holdings” → `Holdings`
  - “Ltd” or “Limited” → `Ltd`
  - Others → `Other`
- Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.
3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range `column).

  Examples:

  * '$8.00-$10.00' → `9.0`

  * '$5.00' → `5.0`

  * '-' → `None`

4. Convert `Shares Offered` to numeric, clean missing or invalid values.
5. Create a new column:
`Withdrawn Value = Shares Offered * Avg Price` (71 non-null values)
6. Group by `Company Class` and calculate total withdrawn value.
7. **Answer**: Which class had the highest total value of withdrawals?

### Step 1

In [3]:
def get_ipos_withdrawal_value() -> pd.DataFrame:
    """
    Fetch IPO data from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No data found.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [4]:
# Load the data and display first 5 rows
df1 = get_ipos_withdrawal_value()
df1.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [6]:
df1.duplicated().sum()

np.int64(0)

### Step 2

In [7]:
def company_class(company_name):
    name = str(company_name).lower()

    # split using any non-alphanumeric separator
    words = re.split(r'\W+', name)

    # Map the company class based on the name
    if "acquisition" in words and ("corp" in words or "corp." in words or "corporation" in words):
        return "Acq.Corp"
    elif "holdings" in words:
        return "Holdings"
    elif "group" in words:
        return "Group"
    elif "inc" in words or "inc." in words or "incorporated" in words:
        return "Inc"
    elif "ltd" in words or "limited" in words:
        return "Ltd"
    else:
        return "Other"

In [8]:
df1['Company Class'] = df1['Company Name'].apply(company_class)
df1.sample(10)

,Symbol,Company Name,Price Range,Shares Offered,Company Class
65,NAVA,"Nava Health MD, Inc.",$6.00,1350000,Inc
50,ELGP,"Elate Group, Inc.",$4.25,1250000,Group
11,JAG,Job Aire Group Inc.,$4.50 - $5.50,2250000,Group
88,LKVA,Lakeview Acquisition Corporation,$10.00,17500000,Acq.Corp
5,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000,Holdings
22,AGII,AgiiPlus Inc.,$6.00 - $7.00,1400000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
67,SUNF,Sunfire Acquisition Corp Limited,$10.00,10000000,Acq.Corp
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
79,VAQB,Vector Acquisition Corporation IV,$10.00,35000000,Acq.Corp


In [9]:
df1['Company Class'].value_counts()

,count
Company Class,
Inc,40
Acq.Corp,21
Ltd,12
Group,12
Holdings,9
Other,6


### Step 3

In [10]:
def range_to_avg(value):

    # Extract numeric values from the string
    numbers = re.findall(r"\d+\.?\d*", str(value))

    if not numbers:
        return None
    elif len(numbers) == 1:
        return float(numbers[0])
    else:
        return (float(numbers[0]) + float(numbers[1])) / 2

In [11]:
df1['Avg. price'] = df1['Price Range'].apply(range_to_avg)
df1.tail()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Group,NaN
99,HCG,hear.com N.V.,$17.00 - $20.00,16220000,Other,18.5


### Step 4

In [12]:
# Convert to numeric and force invalid values to NaN
df1['Shares Offered'] = pd.to_numeric(df1['Shares Offered'], errors='coerce')

In [13]:
stats = df1['Shares Offered'].describe()

# Format in millions and round to 2 decimal places
summary_millions = stats.copy()
summary_millions[1:] = summary_millions[1:] / 1000000
summary_millions = summary_millions.round(2)

# Convert to DataFrame for display
summary_df = pd.DataFrame(summary_millions).rename(columns={"Shares Offered": "Value (in Millions)"})
print(summary_df)

       Value (in Millions)
count                72.00
mean                  9.98
std                  10.48
min                   0.50
25%                   1.58
50%                   3.75
75%                  20.00
max                  45.00


### Step 5

In [14]:
df1['Withdrawn Value'] = df1['Avg. price'] * df1['Shares Offered']
df1.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Holdings,3.5,7000000.0
2,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN,NaN
3,ROTR,"PHI Group, Inc.",-,NaN,Group,NaN,NaN
4,ONE,One Power Company,-,NaN,Other,NaN,NaN


### Step 6

In [114]:
# Group by Company Class and calculate total withdrawn value
grouped_df = df1.groupby('Company Class')['Withdrawn Value'].sum().sort_values(ascending=False).reset_index()

# Convert to millions
grouped_df['Withdrawn Value (Millions)'] = grouped_df['Withdrawn Value'] / 1000000
grouped_df['Withdrawn Value (Millions)'] = grouped_df['Withdrawn Value (Millions)'].map('{:.2f}M'.format)

print(grouped_df[['Company Class', 'Withdrawn Value (Millions)']])

  Company Class Withdrawn Value (Millions)
0      Acq.Corp                   4021.00M
1           Inc                   1912.41M
2         Other                    767.92M
3         Group                    366.54M
4           Ltd                    321.73M
5      Holdings                    315.00M


### Answer

The Acq.Corp class has the highest withdrawn value of 4.02B dollars

## Question 2: [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)
### What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.

### **Steps:**

1. Using the same approach as in Question 1, download the IPOs in 2024 from:
https://stockanalysis.com/ipos/2024/
Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).

  ➤ You should have **75 tickers**.

2. Use Code Snippet 7 to **download daily stock data** for those tickers (via `yfinance`).
Make sure you understand how `growth_1d` ... `growth_365d`, and `volatility` columns are defined.
Define a new column `growth_252d` representing growth after 252 trading days (~1 year), in addition to any other growth periods you already track.

3. **Calculate the Sharpe ratio** assuming a risk-free rate of **4.5%**:

      ` stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility'] `

4. **Filter the DataFrame** to keep data only for the trading day:**2025-06-06**

  Compute descriptive statistics (e.g., `.describe()`) for these columns:

  * `growth_252d`
  * `Sharpe`

  You should observe:

  * `growth_252d` is defined for 71 out of 75 stocks (some IPOs are too recent or data starts later).
  * Median `growth_252d` is approximately 0.75 (indicating a 25% decline), while mean is about 1.15, showing a bias towards high-growth companies pushing the average up.

5. **Answer**:

  * What is the median Sharpe ratio for these 71 stocks?
  * Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.
  * [Additional] Do you observe the same top 10 companies when sorting by `growth_252d` versus sorting by `Sharpe`?


### **Step 1**

In [16]:
def get_ipos_2024() -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/2024/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [17]:
ipos_2024 = get_ipos_2024()
ipos_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$4.82,20.55%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$23.95,498.75%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.60,-85.00%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.26,2.60%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.20,2.00%


In [18]:
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [19]:
missing_prices = ipos_2024[ipos_2024['IPO Price'].astype(str).str.find('-') >= 0]
missing_prices

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
61,"Oct 11, 2024",STFS,Star Fashion Culture Holdings Limited,-,$1.16,-
120,"Jul 23, 2024",ZDAI,Primega Group Holdings Limited,-,$0.93,-
148,"May 31, 2024",NAKA,"Kindly MD, Inc.",-,$13.94,-
174,"Apr 17, 2024",SUPX,SuperX AI Technology Limited,-,$9.63,-


Cleaning

In [20]:
# Remove '$' from 'IPO Price' and 'Current' columns and '%' from Return column
ipos_2024['IPO Price'] = ipos_2024['IPO Price'].str.replace('$', '', regex=False)
ipos_2024['Current'] = ipos_2024['Current'].str.replace('$', '', regex=False)
ipos_2024['Return(%)'] = ipos_2024['Return'].str.replace('%', '', regex=False)

In [21]:
# Convert to numeric
ipos_2024[['IPO Price', 'Current', 'Return(%)']] = ipos_2024[['IPO Price', 'Current', 'Return(%)']].apply(pd.to_numeric, errors='coerce')

# Convert to datetime
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'], format='mixed')

In [22]:
ipos_2024['Return'] = ipos_2024['Current'] - ipos_2024['IPO Price']

In [23]:
ipos_2024.dropna(inplace=True)  # drop missing values since price is important for the purpose of this question

In [24]:
ipos_2024.sample(3)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return,Return(%)
77,2024-09-26,IBG,Innovation Beverage Group Limited,4.0,0.59,-3.41,-85.23
200,2024-02-09,MGX,"Metagenomi, Inc.",15.0,1.48,-13.52,-90.13
205,2024-02-07,AHR,"American Healthcare REIT, Inc.",12.0,35.74,23.74,197.83


In [25]:
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 219 entries, 0 to 224
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      219 non-null    datetime64[ns]
 1   Symbol        219 non-null    object        
 2   Company Name  219 non-null    object        
 3   IPO Price     219 non-null    float64       
 4   Current       219 non-null    float64       
 5   Return        219 non-null    float64       
 6   Return(%)     219 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 13.7+ KB


In [26]:
filtered_ipos_2024 = ipos_2024[ipos_2024['IPO Date'] < '2024-06-01'].reset_index(drop=True)
filtered_ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      75 non-null     datetime64[ns]
 1   Symbol        75 non-null     object        
 2   Company Name  75 non-null     object        
 3   IPO Price     75 non-null     float64       
 4   Current       75 non-null     float64       
 5   Return        75 non-null     float64       
 6   Return(%)     75 non-null     float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 4.2+ KB


In [27]:
filtered_ipos_2024.describe()

,IPO Date,IPO Price,Current,Return,Return(%)
count,75,75.000000,75.000000,75.000000,75.000000
mean,2024-03-16 04:09:36,13.696800,16.583067,2.886267,2.706000
min,2024-01-09 00:00:00,4.000000,0.020000,-49.770000,-99.570000
25%,2024-02-06 12:00:00,4.000000,1.335000,-3.920000,-76.360000
50%,2024-03-21 00:00:00,10.000000,4.650000,-2.300000,-40.400000
75%,2024-04-22 00:00:00,17.750000,20.060000,0.965000,14.900000
max,2024-05-23 00:00:00,92.000000,141.160000,107.160000,849.250000
std,NaN,15.253969,26.602389,20.841133,134.557228


### Step 2

In [28]:
# Extract the symbols in filtered_ipos_2024 into a list
tickers = filtered_ipos_2024['Symbol'].tolist()
print("Tickers:", tickers)

Tickers: ['BOW', 'HDL', 'RFAI', 'JDZG', 'RAY', 'BTOC', 'ZK', 'GPAT', 'PAL', 'SVCO', 'NNE', 'CCIX', 'VIK', 'ZONE', 'LOAR', 'MRX', 'RBRK', 'NCI', 'MFI', 'YYGH', 'TRSG', 'CDTG', 'CTRI', 'IBTA', 'MTEN', 'TWG', 'ULS', 'PACS', 'MNDR', 'CTNM', 'MAMO', 'ZBAO', 'BOLD', 'MMA', 'UBXG', 'IBAC', 'AUNA', 'BKHA', 'LOBO', 'RDDT', 'ALAB', 'INTJ', 'RYDE', 'LGCL', 'SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'HLXB', 'MGX', 'TBBB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO', 'GUTS', 'AS', 'FBLG', 'AVBP', 'BTSG', 'HAO', 'CGON', 'YIBO', 'JL', 'SUGP', 'JVSA', 'KSPI', 'CCTG', 'PSBD', 'SYNX', 'SDHC', 'ROMA']


In [29]:
def get_growth_df(df: pd.DataFrame) -> pd.DataFrame:
    GROWTH_KEYS = []
    for i in [1, 3, 7, 30, 90, 252, 365]:
        col_name = f'growth_{i}d'
        df[col_name] = df['Close'] / df['Close'].shift(i)
        GROWTH_KEYS.append(col_name)

    # Reset index to turn the Date into a column
    df = df.reset_index()
    return df[['Date', 'Symbol', 'Close'] + GROWTH_KEYS]

In [30]:
growth_dfs = []

for ticker in tickers:
    try:
        df = yf.Ticker(ticker).history(period = "max",
                     interval = "1d")[['Close']]

        if not df.empty:
            df['Symbol'] = ticker
            df = get_growth_df(df.copy())
            growth_dfs.append(df)
        else:
            print(f"No data for {ticker}")
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

In [31]:
growth_df = pd.concat(growth_dfs, ignore_index=True)
growth_df.head()

,Date,Symbol,Close,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,growth_365d
0,2024-05-23 00:00:00-04:00,BOW,23.799999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-05-24 00:00:00-04:00,BOW,25.700001,1.079832,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-05-28 00:00:00-04:00,BOW,26.480000,1.030350,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-05-29 00:00:00-04:00,BOW,26.290001,0.992825,1.104622,NaN,NaN,NaN,NaN,NaN
4,2024-05-30 00:00:00-04:00,BOW,26.139999,0.994294,1.017121,NaN,NaN,NaN,NaN,NaN


In [32]:
growth_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23246 entries, 0 to 23245
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype                           
---  ------       --------------  -----                           
 0   Date         23246 non-null  datetime64[ns, America/New_York]
 1   Symbol       23246 non-null  object                          
 2   Close        23246 non-null  float64                         
 3   growth_1d    23171 non-null  float64                         
 4   growth_3d    23021 non-null  float64                         
 5   growth_7d    22721 non-null  float64                         
 6   growth_30d   20996 non-null  float64                         
 7   growth_90d   16496 non-null  float64                         
 8   growth_252d  4430 non-null   float64                         
 9   growth_365d  0 non-null      float64                         
dtypes: datetime64[ns, America/New_York](1), float64(8), object(1)
memory usage: 1.8+ M

In [33]:
# Remove timezone from Date column
growth_df['Date'] = growth_df['Date'].dt.tz_localize(None)

In [34]:
growth_df['volatility'] = growth_df['Close'].rolling(30).std() * np.sqrt(252)
growth_df['volatility'].describe()

,volatility
count,23217.000000
mean,30.255068
std,66.789544
min,0.001957
25%,2.173744
50%,7.888724
75%,27.910156
max,739.764678


### Step 3

In [35]:
# Calculate shrape ratio
growth_df['Sharpe'] = (growth_df['growth_252d'] - 0.045) / growth_df['volatility']
growth_df.tail(3)

,Date,Symbol,Close,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,growth_365d,volatility,Sharpe
23243,2025-06-16,ROMA,2.875,1.080827,0.958333,0.805322,1.955782,4.342900,5.424529,NaN,9.939668,0.541218
23244,2025-06-17,ROMA,2.795,0.972174,0.970486,0.755405,1.814935,4.092240,5.008960,NaN,9.469241,0.524219
23245,2025-06-18,ROMA,2.790,0.998211,1.048872,0.975524,1.516304,4.182909,5.157116,NaN,9.196931,0.555850


### Step 4

In [36]:
# Filter for 2025-06-06
filtered_df = growth_df[growth_df['Date'] == '2025-06-06']
filtered_df

,Date,Symbol,Close,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,growth_365d,volatility,Sharpe
259,2025-06-06,BOW,36.389999,1.010272,0.971696,0.996986,0.879623,1.105407,1.442331,NaN,24.508713,0.057014
531,2025-06-06,HDL,20.410000,0.998532,1.023160,1.039206,0.969366,0.869992,1.007155,NaN,18.816306,0.051134
771,2025-06-06,RFAI,10.510000,1.000000,1.000952,1.002863,1.007960,1.024366,NaN,NaN,0.356043,NaN
1045,2025-06-06,JDZG,0.295000,0.862573,0.565134,0.556604,0.564054,0.393858,0.168571,NaN,1.475159,0.083768
1319,2025-06-06,RAY,1.255000,1.081897,0.774691,0.456364,0.459707,0.980469,0.343459,NaN,9.314924,0.032041
...,...,...,...,...,...,...,...,...,...,...,...,...
21802,2025-06-06,CCTG,1.095000,1.004587,0.969027,0.904959,0.829545,0.655689,0.500000,NaN,1.297728,0.350613
22158,2025-06-06,PSBD,13.880000,1.001443,1.001443,0.993558,1.090338,0.903507,0.947565,NaN,9.712591,0.092927
22517,2025-06-06,SYNX,1.680000,0.976744,0.982456,0.938547,0.908108,0.413793,0.626632,NaN,1.368371,0.425054
22875,2025-06-06,SDHC,19.270000,0.969804,1.017423,1.079552,0.955853,0.775765,0.758065,NaN,16.442821,0.043366


In [37]:
filtered_df[['growth_252d', 'Sharpe']].describe()

,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152897,0.288285
std,1.406017,0.519028
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.311507
max,8.097413,2.835668


### Answer

The median Sharpe ratio for the stocks is 0.0837.

Positive Sharpe means growth exceeding the risk-free rate of 4.5%.

In [38]:
filtered_df[['Symbol', 'growth_252d']].sort_values(by='growth_252d', ascending=False).head(10)

,Symbol,growth_252d
20419,JL,8.097413
23237,ROMA,6.156406
14243,UMAC,4.966533
2948,NNE,4.655224
4632,RBRK,3.184065
16626,AHR,2.483097
17969,AS,2.478203
4344,MRX,2.300384
11479,RDDT,2.225505
6964,MTEN,2.210432


In [39]:
filtered_df[['Symbol', 'Sharpe']].sort_values(by='Sharpe', ascending=False).head(10)

,Symbol,Sharpe
10856,BKHA,2.835668
21091,JVSA,2.041531
16933,LEGT,1.940266
10269,IBAC,1.637119
14583,HLXB,1.123493
8152,MNDR,0.974234
13571,DYCQ,0.969321
12105,INTJ,0.744512
20419,JL,0.566222
5796,TRSG,0.518080


We do not observe the same top 10 companies when sorting by growth_252d versus sorting by Sharpe.

## Question 3: [IPO] ‘Fixed Months Holding Strategy’
### What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?
(Assume you buy at the close of the first trading day and sell after a fixed number of trading days.)

### **Goal:**

Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

### **Steps:**

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).

  Add 12 new columns:
  `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`

  *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*
  This logic is similar to `historyPrices['growth_future_30d']` from Code Snippet 7, but extended to longer timeframes.

2. **Determine the first trading day**
 `(min_date)` for each ticker.
This is the earliest date in the data for each stock.

3. **Join the data:**
Perform an inner join between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.

  ➤ You should end up with 75 records (one per IPO) with all 12 `future_growth_... ` fields populated.

4. **Compute descriptive statistics** for the resulting DataFrame:

  Use `.describe()` or similar to analyze each of the 12 columns:

  * `future_growth_1m`
  * `future_growth_2m`
  * ...
  * `future_growth_12m`

5. **Determine the best holding period:**

  * Find the number of months (1 to 12) where the average (mean) future growth is maximal.
  * This optimal month shows an uplift of >1% compared to all others.
  * Still, the average return remains less than 1 (i.e., expected return is less than doubling your investment).

### Step 1

In [50]:
# Define a function to get future growth features

def get_future_growth(df: pd.DataFrame) -> pd.DataFrame:
    GROWTH_KEYS = []
    for i in range(21, 253, 21):
        col_name = f'future_growth_{i}d'
        df[col_name] = df['Close'].shift(-i) / df['Close']
        GROWTH_KEYS.append(col_name)

    df = df.reset_index()
    return df[['Date', 'Symbol', 'Close'] + GROWTH_KEYS]

In [51]:
future_growth_dfs = []

for ticker in tickers:
    try:
        # Download historical data
        df = yf.Ticker(ticker).history(period="max", interval='1d')[['Close']]

        if not df.empty:
            df['Symbol'] = ticker
            df = get_future_growth(df.copy())
            future_growth_dfs.append(df)
        else:
            print(f"No data for {ticker}")

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

future_growth_df = pd.concat(future_growth_dfs, ignore_index=True)

In [54]:
future_growth_df = pd.concat(future_growth_dfs, ignore_index=True)
future_growth_df.head()

,Date,Symbol,Close,future_growth_21d,future_growth_42d,future_growth_63d,future_growth_84d,future_growth_105d,future_growth_126d,future_growth_147d,future_growth_168d,future_growth_189d,future_growth_210d,future_growth_231d,future_growth_252d
0,2024-05-23 00:00:00-04:00,BOW,23.799999,1.024790,1.139076,1.301681,1.167647,1.239076,1.461765,1.443698,1.357983,1.389076,1.722689,1.698319,1.533613
1,2024-05-24 00:00:00-04:00,BOW,25.700001,0.938132,1.088327,1.196498,1.090661,1.148249,1.392996,1.354086,1.280934,1.276654,1.640078,1.589883,1.445914
2,2024-05-28 00:00:00-04:00,BOW,26.480000,0.955816,1.052492,1.155589,1.048338,1.111027,1.361783,1.327039,1.233761,1.270015,1.535121,1.517749,1.408610
3,2024-05-29 00:00:00-04:00,BOW,26.290001,0.963865,1.058958,1.149106,1.053252,1.139977,1.378851,1.311525,1.248764,1.315709,1.559528,1.521491,1.430582
4,2024-05-30 00:00:00-04:00,BOW,26.139999,0.960214,1.072303,1.198164,1.071538,1.149579,1.383321,1.321729,1.236802,1.303367,1.595639,1.584545,1.432670


In [56]:
# Remove timezone from Date column
future_growth_df['Date'] = future_growth_df['Date'].dt.tz_localize(None)

### Step 2

In [59]:
# Group by Symbol and get the min date
min_date = future_growth_df.groupby('Symbol')['Date'].min().reset_index()
min_date.sample(3)

,Symbol,Date
71,YYGH,2024-04-22
51,RBRK,2024-04-25
65,UBXG,2024-03-28


### Step 3

In [60]:
combined_df = pd.merge(min_date, future_growth_df, on=['Symbol', 'Date'], how='inner')
combined_df.head()

,Symbol,Date,Close,future_growth_21d,future_growth_42d,future_growth_63d,future_growth_84d,future_growth_105d,future_growth_126d,future_growth_147d,future_growth_168d,future_growth_189d,future_growth_210d,future_growth_231d,future_growth_252d
0,AHR,2024-02-07,12.433780,1.044629,1.061430,1.060660,1.125316,1.214961,1.308118,1.753552,1.963886,1.988395,2.257204,2.259752,2.273312
1,ALAB,2024-03-20,62.029999,1.039658,1.168951,1.018217,0.853780,0.631307,0.759794,1.078027,1.457037,1.974367,1.982105,1.381912,1.189424
2,ANRO,2024-02-02,20.700001,0.772947,0.680193,0.686957,0.546377,0.563768,0.438164,0.647343,0.492270,0.192271,0.203865,0.222705,0.202899
3,AS,2024-02-01,13.400000,1.294030,1.172388,1.080597,1.143284,0.935075,0.802239,0.999254,1.236567,1.332090,1.950000,2.152985,2.362687
4,AUNA,2024-03-22,9.600000,0.713542,0.923958,0.801042,0.919792,0.782292,0.767708,0.723958,0.733333,0.706250,0.857292,0.850000,0.752083


In [61]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Symbol              75 non-null     object        
 1   Date                75 non-null     datetime64[ns]
 2   Close               75 non-null     float64       
 3   future_growth_21d   75 non-null     float64       
 4   future_growth_42d   75 non-null     float64       
 5   future_growth_63d   75 non-null     float64       
 6   future_growth_84d   75 non-null     float64       
 7   future_growth_105d  75 non-null     float64       
 8   future_growth_126d  75 non-null     float64       
 9   future_growth_147d  75 non-null     float64       
 10  future_growth_168d  75 non-null     float64       
 11  future_growth_189d  75 non-null     float64       
 12  future_growth_210d  74 non-null     float64       
 13  future_growth_231d  74 non-null     float64       
 

### Step 4

In [62]:
combined_df.describe()

,Date,Close,future_growth_21d,future_growth_42d,future_growth_63d,future_growth_84d,future_growth_105d,future_growth_126d,future_growth_147d,future_growth_168d,future_growth_189d,future_growth_210d,future_growth_231d,future_growth_252d
count,75,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,71.000000
mean,2024-03-24 07:40:48,15.848219,0.927259,0.940749,0.833988,0.825192,0.803849,0.864268,0.847206,0.833046,0.881847,0.917991,0.883057,0.901665
min,2024-01-09 00:00:00,0.011600,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.048274,0.043103,0.033144,0.041357,0.023674,0.038947
25%,2024-02-08 12:00:00,4.140000,0.778984,0.685815,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.226740,0.242424,0.264661,0.229211
50%,2024-03-28 00:00:00,10.040000,0.977000,1.000000,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.772592,0.717585,0.659355
75%,2024-04-25 00:00:00,19.900000,1.046509,1.154013,1.069085,1.134300,1.016381,1.093948,1.114468,1.082365,1.049719,1.200678,1.106735,1.136392
max,2024-09-11 00:00:00,98.000000,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711
std,NaN,18.240347,0.346261,0.574267,0.409480,0.401772,0.488226,0.653079,0.712880,0.762355,0.936894,0.911383,0.862902,0.892658


### Step 5

In [64]:
# Calculate the mean for each future growth column
mean_growths = combined_df[[col for col in combined_df.columns if 'future_growth_' in col]].mean()

# Find the column with the highest mean
optimal_month_col = mean_growths.idxmax()
optimal_month_mean = mean_growths.max()

print("\nColumn with the highest mean:", optimal_month_col)
print("Highest mean growth:", optimal_month_mean)


Column with the highest mean: future_growth_42d
Highest mean growth: 0.9407486074947038


In [65]:
# Check for >1% uplift compared to all others
uplift_check = True
for col, mean_value in mean_growths.items():
    if col != optimal_month_col and optimal_month_mean - mean_value <= 0.01:
        uplift_check = False
        break

print(f"\nUplift > 1% compared to all others: {uplift_check}")
print(f"Average return less than 1: {optimal_month_mean < 1}")


Uplift > 1% compared to all others: True
Average return less than 1: True


## Question 4: [Strategy] Simple RSI-Based Trading Strategy

### What is the total profit (in thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?

### **Goal**:

Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.

### **Steps:**

1. **Run the full notebook from Lecture 2 (33 stocks)**

* Ensure you can generate the merged DataFrame containing:
  - OHLCV data
  - Technical indicators
  - Macro indicators

* Focus on getting RSI computed using Code Snippets 8 and 9.
* This process is essential and will help during the capstone project.

2. ⚠️ **IMPORTANT** Please use this file to solve the Home Assignment (all next steps)

  Download precomputed data using this snippet:

  ```
  import gdown
  import pandas as pd

  file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
  gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
  df = pd.read_parquet("data.parquet", engine="pyarrow")
  ```

3. **RSI Strategy Setup:**

* RSI is already available in the dataset as a field.
* The threshold for oversold is defined as `RSI < 25`.

4. Filter the dataset by RSI and date:

  rsi_threshold = 25

  selected_df = df[
      (df['rsi'] < rsi_threshold) &
      (df['Date'] >= '2000-01-01') &
      (df['Date'] <= '2025-06-01')
  ]

5. **Calculate Net Profit Over 25 Years:**

* Total number of trades: 1568

* For each trade, you invest $1000

* Use the 30-day forward return (`growth_future_30d`) to compute net earnings:

  ```
  net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
  ```
  
* **Final Answer:**

  **What is the net income in $K** (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?

### Step 1

OHLCV data

Technical indicators

Macro indicators

### Step 2

In [66]:
import gdown

In [85]:
# Download precomputed data
file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=ecc36f45-a434-44c6-87c9-82be7fc8aac4
To: /content/data.parquet
100%|██████████| 130M/130M [00:00<00:00, 225MB/s]


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229932 entries, 0 to 5690
Columns: 203 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(129), int32(64), int64(5), object(2)
memory usage: 301.7+ MB


### Step 3

In [68]:
df.head()

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.054277,0.062259,0.054277,0.059598,1.031789e+09,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.059598,0.062791,0.059598,0.061726,3.081600e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.061726,0.063323,0.061726,0.062791,1.331712e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.062791,0.063323,0.060662,0.061194,6.776640e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.061194,0.061726,0.059598,0.060130,4.789440e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df['rsi'].sample(5)

,rsi
4847,66.887907
6222,59.690241
597,57.957606
1683,56.364789
4419,60.943841


### Step 4

In [86]:
# Filter using the threshold and date
rsi_threshold = 25

selected_df = df[ (df['rsi'] < rsi_threshold) & (df['Date'] >= '2000-01-01') & (df['Date'] <= '2025-06-01') ].reset_index()

In [89]:
selected_df.shape

(1568, 204)

### Step 5

In [108]:
close_cols = selected_df [[col for col in selected_df .columns if 'Close_' in col]]
close_cols

,Close_x,Close_y
0,19.673643,18.520000
1,19.309669,20.250000
2,17.336559,21.850000
3,16.991741,21.540001
4,16.972589,21.030001
...,...,...
1563,769.289795,72.000000
1564,788.998840,66.040001
1565,660.662170,61.590000
1566,654.971985,61.669998


In [115]:
# 30-day forward return
selected_df['growth_future_30d'] = selected_df['Close_x'].shift(-30) / selected_df['Close_x']

In [116]:
# Calculate net income
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
print(f"Net income in $K: {net_income / 1000:.0f}")

Net income in $K: 10769
